In [1]:
data = (r"/content/laptops_dataset_final_600.csv")

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(data)

In [3]:
df.tail()

,product_name,overall_rating,no_ratings,no_reviews,rating,title,review
24108,MSI Modern 14 Intel Core i5 13th Gen 1335U - (...,4.3,156,24,5,Perfect product!,MSI Laptop is high performance and the best. c...
24109,MSI Modern 14 Intel Core i5 13th Gen 1335U - (...,4.3,156,24,5,Perfect product!,Excellent performance best laptop.
24110,MSI Modern 14 Intel Core i5 13th Gen 1335U - (...,4.3,156,24,4,"Good product, Lacks features.",Decent battery life. Exceptional build quality...
24111,Lenovo IdeaPad 5 2-in-1 WUXGA IPS AMD Ryzen 7 ...,4.4,7,2,3,Nice,The product does not support facial recognitio...
24112,Lenovo IdeaPad 5 2-in-1 WUXGA IPS AMD Ryzen 7 ...,4.4,7,2,4,Very Good,DISPLAY IS A LET DOWN. But the lenovo Pen work...


In [4]:
df_new = df.drop(columns=['overall_rating','no_ratings','no_reviews','rating','title'])
df_new

,product_name,review
0,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,"Loved it, it's my first MacBook that I earned ..."
1,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,Battery lasted longer than my first relationsh...
2,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,Such a great deal.. very happy with the perfor...
3,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,"Awesome build quality and very good display, b..."
4,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,When i ordered and came to know about seller r...
...,...,...
24108,MSI Modern 14 Intel Core i5 13th Gen 1335U - (...,MSI Laptop is high performance and the best. c...
24109,MSI Modern 14 Intel Core i5 13th Gen 1335U - (...,Excellent performance best laptop.
24110,MSI Modern 14 Intel Core i5 13th Gen 1335U - (...,Decent battery life. Exceptional build quality...
24111,Lenovo IdeaPad 5 2-in-1 WUXGA IPS AMD Ryzen 7 ...,The product does not support facial recognitio...


In [5]:
import re

def clean_text(text):
  """Removes brackets, forward slashes, and multiple dots from text."""
  text = re.sub(r"[\[\]\(\)]", "", text)  # Remove brackets
  text = re.sub(r"/", "", text)  # Remove forward slashes
  text = re.sub(r"\.{2,}", ".", text)  # Replace multiple dots with a single dot
  return text

In [6]:
df_new['product_name'] = df_new['product_name'].apply(clean_text)
df_new

,product_name,review
0,Apple MacBook AIR Apple M2 - 8 GB256 GB SSDMac...,"Loved it, it's my first MacBook that I earned ..."
1,Apple MacBook AIR Apple M2 - 8 GB256 GB SSDMac...,Battery lasted longer than my first relationsh...
2,Apple MacBook AIR Apple M2 - 8 GB256 GB SSDMac...,Such a great deal.. very happy with the perfor...
3,Apple MacBook AIR Apple M2 - 8 GB256 GB SSDMac...,"Awesome build quality and very good display, b..."
4,Apple MacBook AIR Apple M2 - 8 GB256 GB SSDMac...,When i ordered and came to know about seller r...
...,...,...
24108,MSI Modern 14 Intel Core i5 13th Gen 1335U - 8...,MSI Laptop is high performance and the best. c...
24109,MSI Modern 14 Intel Core i5 13th Gen 1335U - 8...,Excellent performance best laptop.
24110,MSI Modern 14 Intel Core i5 13th Gen 1335U - 8...,Decent battery life. Exceptional build quality...
24111,Lenovo IdeaPad 5 2-in-1 WUXGA IPS AMD Ryzen 7 ...,The product does not support facial recognitio...


In [7]:
!pip install emoji

In [8]:
import emoji
def remove_emoji(text):
  return emoji.replace_emoji(text, replace='')



In [9]:
df_new['review'] = df_new['review'].apply(remove_emoji)
df_new['review'][0]

"Loved it, it's my first MacBook that I earned from my hardwork "

In [10]:
!pip install transformers datasets torch pandas

In [11]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

In [12]:
import torch

# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please enable GPU in Colab runtime.")

Using device: cuda


In [16]:
corpus = []
for index, row in df_new.iterrows():
    corpus.append({
        'product_name': row['product_name'],
        'review': row['review']
    })


In [17]:
# Save corpus
import json
with open("review_corpus.json", "w") as f:
    json.dump(corpus, f, indent=2)

In [18]:
# Convert to Hugging Face Dataset
from datasets import Dataset
hf_dataset = Dataset.from_list(corpus)

In [19]:
# Load tokenizer and model
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [20]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [28]:
# Tokenize dataset
def tokenize_function(examples):
    text = [f"{inp} {tokenizer.eos_token} {resp}" for inp, resp in zip(examples['product_name'], examples['review'])]
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
        return_attention_mask=True  # Explicitly return attention_mask
    )
    tokenized['labels'] = tokenized['input_ids'].clone()
    return {k: v.to(device) for k, v in tokenized.items()}  # Move to CUDA



In [29]:
# Tokenize with batched processing
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['product_name', 'review'])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/24113 [00:00<?, ? examples/s]

In [31]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./flipkart_chatbot",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True ,
    dataloader_pin_memory=True,
)


In [32]:
pip install wandb

In [34]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: siddharthkushwaha33 (siddharthkushwaha33-gec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [35]:
# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: siddharthkushwaha33 (siddharthkushwaha33-gec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,5.996100
20,5.166200
30,4.673100
40,3.272500
50,2.787000
60,2.356500
70,1.481400
80,2.193800
90,1.543100
100,1.581000


TrainOutput(global_step=18087, training_loss=0.5868621042937777, metrics={'train_runtime': 2503.0919, 'train_samples_per_second': 28.9, 'train_steps_per_second': 7.226, 'total_flos': 4725401075712000.0, 'train_loss': 0.5868621042937777, 'epoch': 3.0})

In [36]:
model.save_pretrained("./flipkart_chatbot_model")
tokenizer.save_pretrained("./flipkart_chatbot_model")

('./flipkart_chatbot_model/tokenizer_config.json',
 './flipkart_chatbot_model/special_tokens_map.json',
 './flipkart_chatbot_model/vocab.json',
 './flipkart_chatbot_model/merges.txt',
 './flipkart_chatbot_model/added_tokens.json',
 './flipkart_chatbot_model/tokenizer.json')

In [37]:
model = AutoModelForCausalLM.from_pretrained("./flipkart_chatbot_model").to(device)
tokenizer = AutoTokenizer.from_pretrained("./flipkart_chatbot_model")

In [38]:
# Chatbot response function
def chatbot_response(input_text):
    inputs = tokenizer(
        input_text + tokenizer.eos_token,
        return_tensors="pt",
        return_attention_mask=True  # Include attention_mask
    ).to(device)  # Move to CUDA
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],  # Pass attention_mask
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False).split(tokenizer.eos_token)[1].strip()
    return response

In [45]:
test_inputs = """How much ram does Lenovo has?"""

chatbot_response(test_inputs)

'Good one'